# COVID-Team-1
### CSC 405 01
### Last Updated: 09/19/2020


### Module Imports

In [ ]:
import pandas as pd

### Module's Docstring

In [ ]:
"""Gives streamlined access to data paths and dataframes regarding Covid data."""

### Data paths

In [ ]:
# Primary data paths.
_covid_cases_path = "../data/Covid/covid_confirmed_usafacts.csv"
_covid_population_path = "../data/Covid/covid_county_population_usafacts.csv"
_covid_death_path = "../data/Covid/covid_deaths_usafacts.csv"
_covid_primary_data_path = "../data/Covid/covid_primary_data.csv"

# Extension data paths.
_county_employment_path = "../data/Employment/county-ests-employment-and-wages.csv"
_county_hospital_path = "../data/Hospitals/Definitive_Healthcare _USA_Hospital_Beds.csv"
_housing_demographics_path = "../data/Housing/ACSDP1Y2018.DP05_data_with_overlays_2020-09-03T141858.csv"

### Getters for Unmerged Data

In [ ]:
# Reads in the cases data and adds the prefix "cases -" to ensure date attributes are distinct for merging.
def get_covid_by_cases():
    """Returns dataframe containing data on Covid cases."""
    df_cases = pd.read_csv(_covid_cases_path)
    df_cases = df_cases.add_prefix("cases - ")
    df_cases.rename(columns = {"cases - countyFIPS": "countyFIPS", \
                               "cases - County Name": "County Name", \
                               "cases - State": "State", \
                               "cases - stateFIPS": "stateFIPS"}, \
                    inplace = True,
                   )
    return df_cases

# Reads in the deaths data and adds the prefix "deaths -" to ensure date attributes are distinct for merging.
def get_covid_by_deaths():
    """Returns dataframe containing data on Covid deaths."""
    df_deaths = pd.read_csv(_covid_death_path)
    df_deaths = df_deaths.add_prefix("deaths - ")
    df_deaths.rename(columns = {"deaths - countyFIPS": "countyFIPS", \
                                "deaths - County Name": "County Name", \
                                "deaths - State": "State", \
                                "deaths - stateFIPS": "stateFIPS"}, \
                     inplace = True,
                    )
    return df_deaths

def get_county_population():
    """Returns dataframe containing data on population."""
    return pd.read_csv(_covid_population_path)

def get_county_employment():
    """Returns dataframe containing data on employment and wages."""
    return pd.read_csv(_county_employment_path)

def get_county_hospital():
    """Returns dataframe containing data on hopsital statistics."""
    return pd.read_csv(_county_hospital_path)

### Getter for Merged Data

In [ ]:
# Checks for file existence and returns, or creates file and returns.
def get_covid_primary_data():
    """Returns dataframe containing the primary Covid data (cases, population, death)."""
    try:
        return pd.read_csv(_covid_primary_data_path)
    except:
        covid_primary_data_to_csv()
        return pd.read_csv(_covid_primary_data_path)

### Merged Data to CSV

In [ ]:
# Removes all countyFIPS values of 0, these values are not specific to any county.
def covid_primary_data_to_csv():
    """Merges the primary Covid data (cases, population, death) and outputs it to csv file."""
    first_merge = pd.merge(get_covid_by_cases(), \
                           get_covid_by_deaths(), \
                           how = "inner", \
                           on = "countyFIPS", \
                          )

    second_merge = pd.merge(first_merge, \
                            get_county_population(), \
                            how = "inner",
                            on = "countyFIPS",
                           )
    second_merge = second_merge[second_merge.countyFIPS != 0]
    second_merge.to_csv(_covid_primary_data_path, index=False)